# Question 2 Workflow
## Imports

In [112]:
## change notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [113]:
# Import pandas and biothings explorers modules
import pandas as pandas
from biothings_explorer.query.predict import Predict
from biothings_explorer.query.visualize import display_graph
from biothings_explorer.hint import Hint
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline
import warnings
warnings.filterwarnings("ignore") 

## Set Parameters

In [114]:
max_direct = 20
max_indirect = 50

In [21]:
disease_name = 'cystic fibrosis'
ht = Hint()
disease = ht.query(disease_name)['Disease'][0]
disease

{'MONDO': 'MONDO:0009061',
 'DOID': 'DOID:1485',
 'UMLS': 'C0010674',
 'name': 'cystic fibrosis',
 'MESH': 'D003550',
 'OMIM': '219700',
 'ORPHANET': '586',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0009061'},
 'display': 'MONDO(MONDO:0009061) DOID(DOID:1485) OMIM(219700) ORPHANET(586) UMLS(C0010674) MESH(D003550) name(cystic fibrosis)',
 'type': 'Disease'}

## Get Symptoms

In [30]:
# create dictionary of symptom HPIDs, nad symptom names (with synonyms)
fc = FindConnection(input_obj=cf, output_obj='PhenotypicFeature', intermediate_nodes=None)
fc.connect(verbose=False)
disease_to_phenotypicFeature = fc.display_table_view()
disease_to_phenotypicFeature
symptom_dict = {}
for index, row in disease_to_phenotypicFeature.iterrows():
    if(disease_to_phenotypicFeature['pred1_source'][index] == "hpo"):
        output_name = disease_to_phenotypicFeature['output_name'][index]
        items = fc.fc.G[disease_name][output_name].values()
        for item in items: 
    #         print(item)
            if('frequency' in item['info']):
                freq = [_item['info']['frequency'] for _item in fc.fc.G[disease_name][output_name].values() if "frequency" in _item["info"]][0][0]
                freq_value = ht.query(freq)['PhenotypicFeature'][0]['name']
            else: 
                freq_value = 'Unknown'
            try:             
                symptom_dict[fc.fc.display_node_info(output_name)['equivalent_ids']['HP'][0]] = {
                    "names": fc.fc.display_node_info(output_name)['equivalent_ids']['name'],
                    "frequency": freq_value,
                }
            except: 
                pass
print(symptom_dict)
        
# create list of HPIDs and Symptoms for later use
disease_symptom_hpids = list(symptom_dict.keys())
disease_symptoms = []
for entry in list(symptom_dict.values()): disease_symptoms = disease_symptoms + entry['names']
    
## show dataframe of symptoms
disease_symptom_df = pandas.DataFrame.from_dict(symptom_dict, orient='index')
disease_symptom_df = pandas.concat([disease_symptom_df[disease_symptom_df["frequency"] == "Very frequent"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Frequent"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Occasional"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Rare"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Unknown"]
              ])
disease_symptom_df

{'HP:0004401': {'names': ['DISTAL INTESTINAL OBSTRUCTION SYNDROME', 'MECONIUM ILEUS', 'MECONIUM ILEUS IN NEONATES', 'MECONIUM ILEUS ON ULTRASONOGRAPHY'], 'frequency': 'Unknown'}, 'HP:0002613': {'names': ['BILIARY CIRRHOSIS', 'PRIMARY BILIARY CIRRHOSIS'], 'frequency': 'Unknown'}, 'HP:0002150': {'names': ['ELEVATED URINE CALCIUM LEVELS', 'HYPERCALCINURIA', 'HYPERCALCIURIA'], 'frequency': 'Unknown'}, 'HP:0002110': {'names': ['BRONCHIECTASIS', 'PERMANENT ENLARGEMENT OF THE AIRWAYS OF THE LUNGS'], 'frequency': 'Unknown'}, 'HP:0002099': {'names': ['ASTHMA', 'BRONCHIAL ASTHMA', 'REACTIVE AIRWAY DISEASE'], 'frequency': 'Unknown'}, 'HP:0002035': {'names': ['RECTAL PROLAPSE', 'RECTAL PROLAPSED', 'RECTUM PROTRUDES THROUGH ANUS'], 'frequency': 'Unknown'}, 'HP:0001738': {'names': ['EXOCRINE PANCREATIC INSUFFICIENCY', 'INABILITY TO PROPERLY DIGEST FOOD DUE TO LACK OF PANCREATIC DIGESTIVE ENZYMES', 'PANCREATIC INSUFFICIENCY'], 'frequency': 'Frequent'}, 'HP:0003251': {'names': ['MALE INFERTILITY'], 'f

,names,frequency
HP:0001738,"[EXOCRINE PANCREATIC INSUFFICIENCY, INABILITY ...",Frequent
HP:0001944,[DEHYDRATION],Occasional
HP:0004401,"[DISTAL INTESTINAL OBSTRUCTION SYNDROME, MECON...",Unknown
HP:0002613,"[BILIARY CIRRHOSIS, PRIMARY BILIARY CIRRHOSIS]",Unknown
HP:0002150,"[ELEVATED URINE CALCIUM LEVELS, HYPERCALCINURI...",Unknown
HP:0002110,"[BRONCHIECTASIS, PERMANENT ENLARGEMENT OF THE ...",Unknown
HP:0002099,"[ASTHMA, BRONCHIAL ASTHMA, REACTIVE AIRWAY DIS...",Unknown
HP:0002035,"[RECTAL PROLAPSE, RECTAL PROLAPSED, RECTUM PRO...",Unknown
HP:0003251,[MALE INFERTILITY],Unknown
HP:0001648,[COR PULMONALE],Unknown


## Disease -> Gene -> Phenotypic Feature

In [95]:
pd = Predict(
    input_objs=[disease],
    intermediate_nodes =['Gene'], 
    output_types =['PhenotypicFeature'], 
    config={"expand": True}
)
pd.connect(verbose=True)


========== QUERY EXPANSION ==========

You have selected to expand your inputs.
1 unique bioentities have been found as subclass of your inputs.

========== QUERY PARAMETER SUMMARY ==========

Your query have 2 input nodes, including cystic fibrosis,cystic fibrosis associated meconium ileus .... And BTE will find paths that connect your input nodes to your output types ['PhenotypicFeature']. Paths will contain 1 intermediate nodes.

Intermediate node #1 will have these type constraints: Gene

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Disease
Output Types: Gene
Predicates: None

BTE found 12 APIs based on SmartAPI Meta-KG.

API 1. TCGA Mutation Frequency KP API (2 API calls)
API 2. Automat CORD19 Scigraph API (2 API calls)
API 3. mydisease.info API (1 API calls)
API 4. SEMMED Disease API (15 API calls)
API 5. MGIgene2phenotype API (1 API calls)
API 6. Automat CORD19 Scibite API (2 API calls)
API 7. Automat PHAROS API (2 API calls)
API 8. BioL

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 46, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


API 8.3 BioLink API: 13 hits
API 4.3: https://biothings.ncats.io/semmed/query?fields=affected_by (POST -d q=C0010674&scopes=umls)
API 4.3 SEMMED Disease API: 0 hits
API 8.4: https://api.monarchinitiative.org/api/bioentity/disease/MONDO:0009061/genes?direct=True&rows=200&unselect_evidence=True
API 8.4 BioLink API: 0 hits
API 10.1: https://translator.broadinstitute.org/molepro_reasoner/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'related_to'}], 'nodes': [{'curie': 'MONDO:0005413', 'id': 'n00', 'type': 'disease'}, {'id': 'n01', 'type': 'gene'}]}}})
API 10.1 Molecular Data Provider API: 0 hits
API 10.2: https://translator.broadinstitute.org/molepro_reasoner/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'related_to'}], 'nodes': [{'curie': 'MONDO:0009061', 'id': 'n00', 'type': 'disease'}, {'id': 'n01', 'type': 'gene'}]}}})
API 10.2 Molecular Data Provi

API 4.1 EBIgene2phenotype API: 1460 hits
API 3.1: https://biothings.ncats.io/semmedgene/query?fields=disrupts (POST -d q=C1708411,C1823600,C0919516,C1420498,C1538714,C0165519,C1414504,C3537219,C1416578,C1705962,C1414440,C1517488,C1422211,C0033634,C1425478,C1415585,C1332820,C1412581,C1334093,C0079460,C1539876,C3831460,C1417123,C1705513,C1412071,C1413452,C1424783,C1332124,C1420708,C1367721,C1425602,C1415714,C1708837,C1417811,C1427293,C0070788,C1413681,C1413844,C2982012,C1539907,C1413336,C1423548,C1412260,C1413904,C1413454,C1414083,C2697654,C1825549,C1412622,C1540171,C0011519,C1422804,C1367477,C1414560,C3539699,C1826448,C1413752,C1705982,C1710595,C1367714,C1412077,C0301810,C1413167,C0812246,C1122373,C1412294,C1336930,C1415481,C0170521,C1422675,C0919550,C1420854,C1415754,C1423498,C0025543,C1842747,C1417530,C0015518,C0669365,C0024375,C1413243,C1422673,C1825534,C1333213,C1423607,C1419871,C1413043,C1423108,C0131875,C1415336,C1419129,C1825293,C1421224,C1414266,C1335144,C1415576,C1334126,C09194

API 3.1: https://biothings.ncats.io/semmedgene/query?fields=negatively_regulates (POST -d q=C1708411,C1823600,C0919516,C1420498,C1538714,C0165519,C1414504,C3537219,C1416578,C1705962,C1414440,C1517488,C1422211,C0033634,C1425478,C1415585,C1332820,C1412581,C1334093,C0079460,C1539876,C3831460,C1417123,C1705513,C1412071,C1413452,C1424783,C1332124,C1420708,C1367721,C1425602,C1415714,C1708837,C1417811,C1427293,C0070788,C1413681,C1413844,C2982012,C1539907,C1413336,C1423548,C1412260,C1413904,C1413454,C1414083,C2697654,C1825549,C1412622,C1540171,C0011519,C1422804,C1367477,C1414560,C3539699,C1826448,C1413752,C1705982,C1710595,C1367714,C1412077,C0301810,C1413167,C0812246,C1122373,C1412294,C1336930,C1415481,C0170521,C1422675,C0919550,C1420854,C1415754,C1423498,C0025543,C1842747,C1417530,C0015518,C0669365,C0024375,C1413243,C1422673,C1825534,C1333213,C1423607,C1419871,C1413043,C1423108,C0131875,C1415336,C1419129,C1825293,C1421224,C1414266,C1335144,C1415576,C1334126,C0919432,C1332765,C1416566,C1705879

...

API 2.371: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3329/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.371 BioLink API: 30 hits
API 2.372: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:2706/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.372 BioLink API: 81 hits
API 2.373: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:427/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.373: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:5888/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.373 BioLink API: 98 hits
API 2.374 BioLink API: 120 hits
API 2.375: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:65010/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.375 BioLink API: 0 hits
API 2.376: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:2919/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.376 BioLink API: 0 hits
API 2.377: https:

API 2.423 BioLink API: 0 hits
API 2.424: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7189/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.424 BioLink API: 15 hits
API 2.425: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:9075/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.425 BioLink API: 0 hits
API 2.426: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3684/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.426 BioLink API: 0 hits
API 2.427: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:259295/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.427 BioLink API: 0 hits
API 2.428: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:55733/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.428 BioLink API: 27 hits
API 2.429: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7421/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.429 BioLink

API 2.475: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10914/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.475 BioLink API: 0 hits
API 2.476: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10657/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.476 BioLink API: 0 hits
API 2.477: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10884/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.477 BioLink API: 0 hits
API 2.478: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1084/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.478 BioLink API: 0 hits
API 2.479: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1056/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.479 BioLink API: 27 hits
API 2.480: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10961/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.480 BioLink API: 0 hits
API 2.481: https:

API 2.527 BioLink API: 0 hits
API 2.528: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:340719/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.528 BioLink API: 12 hits
API 2.529: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:355/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.529 BioLink API: 75 hits
API 2.530: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3933/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.530 BioLink API: 0 hits
API 2.531: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3683/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.531 BioLink API: 0 hits
API 2.532: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:4760/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.532 BioLink API: 25 hits
API 2.533: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:4482/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.533: https:/

API 2.579: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:8878/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.579 BioLink API: 132 hits
API 2.580: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10400/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.580 BioLink API: 0 hits
API 2.581: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:4023/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.581: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:973/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.581: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:27165/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.581 BioLink API: 0 hits
API 2.582 BioLink API: 20 hits
API 2.583 BioLink API: 32 hits
API 2.584: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1514/phenotypes?direct=True&rows=200&unselect_evidence=True
API 2.584 BioLink API: 0 hits
API 2.585: https:

In [96]:
df = pd.display_table_view()
df

,input_id,input_label,input_type,pred1,pred1_source,pred1_api,pred1_publications,node1_id,node1_label,node1_type,node1_degree,pred2,pred2_source,pred2_api,pred2_publications,output_id,output_label,output_type,output_degree
0,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:831,CAST,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0022596,Palmoplantar keratoderma,PhenotypicFeature,None
1,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:831,CAST,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0870082,Hyperkeratosis,PhenotypicFeature,None
2,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:831,CAST,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0085661,Onycholysis,PhenotypicFeature,None
3,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:831,CAST,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0240182,Leukonychia,PhenotypicFeature,None
4,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:831,CAST,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0237849,Scaling skin,PhenotypicFeature,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,PMID:18976645,UMLS:C0017837,UMLS:C0017837,Gene,None,negatively_regulates,SEMMED,SEMMED Gene API,"PMID:16521106,PMID:20333650,PMID:21787648,PMID...",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,None
18480,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,PMID:18976645,UMLS:C0017837,UMLS:C0017837,Gene,None,physically_interacts_with,SEMMED,SEMMED Gene API,"PMID:12022988,PMID:21342130,PMID:23623073,PMID...",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,None
18481,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,PMID:18976645,UMLS:C0017837,UMLS:C0017837,Gene,None,positively_regulated_by,SEMMED,SEMMED Gene API,"PMID:10922148,PMID:15087171,PMID:15885683,PMID...",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,None
18482,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,PMID:18976645,UMLS:C0017837,UMLS:C0017837,Gene,None,positively_regulates,SEMMED,SEMMED Gene API,"PMID:28774551,PMID:8481892",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,None


In [97]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df['output_label'])]) if val in disease_symptoms]
direct_relevant_genes_to_symptoms_df = df.iloc[indices_with_symptom_outputs]
direct_relevant_genes_to_symptoms_df

,input_id,input_label,input_type,pred1,pred1_source,pred1_api,pred1_publications,node1_id,node1_label,node1_type,node1_degree,pred2,pred2_source,pred2_api,pred2_publications,output_id,output_label,output_type,output_degree
28,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:5805,PTS,Gene,None,related_to,EBI,EBIgene2phenotype API,"PMID:8178819,PMID:9450907,PMID:10220141",UMLS:C0441748,Autosomal recessive inheritance,PhenotypicFeature,None
96,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:326,AIRE,Gene,None,related_to,EBI,EBIgene2phenotype API,PMID:9398839,UMLS:C0441748,Autosomal recessive inheritance,PhenotypicFeature,None
276,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:1080,CFTR,Gene,None,related_to,"omim,coriell,orphanet,clinvar,hpoa",BioLink API,"PMID:21909392,PMID:10077727,PMID:2236053,PMID:...",UMLS:C0030293,Exocrine pancreatic insufficiency,PhenotypicFeature,None
277,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:1080,CFTR,Gene,None,related_to,"omim,coriell,orphanet,clinvar,hpoa",BioLink API,"PMID:21909392,PMID:10077727,PMID:2236053,PMID:...",UMLS:C0006267,Bronchiectasis,PhenotypicFeature,None
278,MONDO:0009061,cystic fibrosis,Disease,related_to,scigraph,Automat CORD19 Scigraph API,None,NCBIGene:1080,CFTR,Gene,None,related_to,"omim,coriell,orphanet,clinvar,hpoa",BioLink API,"PMID:21909392,PMID:10077727,PMID:2236053,PMID:...",UMLS:C0021364,Male infertility,PhenotypicFeature,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18070,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,"PMID:11802244,PMID:9694345",NCBIGene:10682,EBP,Gene,None,related_to,EBI,EBIgene2phenotype API,"PMID:10391218,PMID:10391219,PMID:10942423,PMID...",UMLS:C0231246,Failure to thrive,PhenotypicFeature,None
18108,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,"PMID:11802244,PMID:9694345",NCBIGene:10682,EBP,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0231246,Failure to thrive,PhenotypicFeature,None
18267,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,PMID:25124375,NCBIGene:182,JAG1,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0030293,Exocrine pancreatic insufficiency,PhenotypicFeature,None
18279,MONDO:0009061,cystic fibrosis,Disease,treated_by,SEMMED,SEMMED Disease API,PMID:25124375,NCBIGene:182,JAG1,Gene,None,related_to,"omim,orphanet,hpoa",BioLink API,None,UMLS:C0231246,Failure to thrive,PhenotypicFeature,None


In [100]:
direct_genes = 	list(direct_relevant_genes_to_symptoms_df["node1_label"])
direct_gene_dict = {x:direct_genes.count(x) for x in direct_genes}
direct_gene_dict = dict(sorted(direct_gene_dict.items(), key=lambda x: x[1], reverse=True)[:max_direct])
direct_gene_dict

{'CFTR': 180,
 'TGFB1': 90,
 'DCTN4': 75,
 'HFE': 45,
 'STX1A': 30,
 'CLCA4': 30,
 'SCNN1B': 20,
 'SCNN1A': 16,
 'SCNN1G': 15,
 'MTHFR': 15,
 'ADA': 12,
 'INS': 9,
 'SLC26A3': 6,
 'CA2': 6,
 'MMAA': 5,
 'UBR1': 5,
 'SLC26A2': 4,
 'LEP': 4,
 'SMPD1': 4,
 'ASAH1': 4}

## Disease X -> Intermediate Node -> Gene -> Phenotypic Feature

In [101]:
query_config = {
    "expand": True
}

In [102]:
pd2 = Predict(
    input_objs=[disease],
    intermediate_nodes =[['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'],'Gene'], 
    output_types =['PhenotypicFeature'], 
    config= query_config 
)
pd2.connect(verbose=True)


========== QUERY EXPANSION ==========

You have selected to expand your inputs.
1 unique bioentities have been found as subclass of your inputs.

========== QUERY PARAMETER SUMMARY ==========

Your query have 2 input nodes, including cystic fibrosis,cystic fibrosis associated meconium ileus .... And BTE will find paths that connect your input nodes to your output types ['PhenotypicFeature']. Paths will contain 2 intermediate nodes.

Intermediate node #1 will have these type constraints: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Intermediate node #2 will have these type constraints: Gene

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Disease
Output Types: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'Phenotyp

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 46, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Ver

API 9.3 BioLink API: 0 hits
API 9.4: https://api.monarchinitiative.org/api/bioentity/disease/MONDO:0009061/genes?direct=True&rows=200&unselect_evidence=True
API 9.4 BioLink API: 13 hits
API 2.1: https://automat.renci.org/cord19-scigraph/disease/chemical_substance/MONDO:0005413
API 2.1 Automat CORD19 Scigraph API: 0 hits
API 8.1: https://automat.renci.org/pharos/disease/gene/MONDO:0005413
API 8.1 Automat PHAROS API: 0 hits
API 12.1: https://automat.renci.org/hmdb/disease/chemical_substance/MONDO:0009061
API 12.1 Automat HMDB API: 1 hits
API 7.1: https://automat.renci.org/cord19-scibite/disease/gene/MONDO:0005413
API 7.1 Automat CORD19 Scibite API: 0 hits
API 12.2: https://automat.renci.org/hmdb/disease/chemical_substance/MONDO:0009061
API 12.2 Automat HMDB API: 20 hits
API 8.2: https://automat.renci.org/pharos/disease/gene/MONDO:0009061
API 8.2 Automat PHAROS API: 44 hits
API 12.3: https://automat.renci.org/hmdb/disease/chemical_substance/MONDO:0005413
API 12.3 Automat HMDB API: 0 hits


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 46, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Ver

API 12.4 Automat HMDB API: 0 hits
API 13.1: https://translator.broadinstitute.org/molepro_reasoner/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'related_to'}], 'nodes': [{'curie': 'MONDO:0005413', 'id': 'n00', 'type': 'disease'}, {'id': 'n01', 'type': 'gene'}]}}})
API 13.1 Molecular Data Provider API: 0 hits
API 13.2: https://translator.broadinstitute.org/molepro_reasoner/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'related_to'}], 'nodes': [{'curie': 'MONDO:0009061', 'id': 'n00', 'type': 'disease'}, {'id': 'n01', 'type': 'gene'}]}}})
API 13.2 Molecular Data Provider API: 0 hits
API 14.1: https://translator.broadinstitute.org/genetics_data_provider/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'associated'}], 'nodes': [{'curie': 'MONDO:0009061', 'id': 'n00', 'type': 'disease'},

...

API 2.1 BioLink API: 65 hits
API 2.2: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:727897/interactions?direct=True&rows=200&unselect_evidence=True
API 22.1: https://automat.renci.org/hmdb/chemical_substance/gene/CHEBI:81281
API 22.1 Automat HMDB API: 0 hits
API 22.2: https://automat.renci.org/hmdb/chemical_substance/gene/CHEBI:57921
API 22.2 Automat HMDB API: 0 hits
API 23.1: https://automat.renci.org/chembio/chemical_substance/gene/CHEBI:57921
API 23.1 Automat CHEMBIO API: 0 hits
API 22.3: https://automat.renci.org/hmdb/chemical_substance/gene/CHEBI:86404
API 22.3 Automat HMDB API: 0 hits
API 23.2: https://automat.renci.org/chembio/chemical_substance/gene/CHEBI:81281
API 23.2 Automat CHEMBIO API: 0 hits
API 23.3: https://automat.renci.org/chembio/chemical_substance/gene/CHEBI:86404
API 23.3 Automat CHEMBIO API: 0 hits
API 23.4: https://automat.renci.org/chembio/chemical_substance/gene/CHEBI:16844
API 23.4 Automat CHEMBIO API: 0 hits
API 22.4: https://automat.renci.org

API 20.1 MyChem.info API: 749 hits
API 20.2: https://mychem.info/v1/query?fields=drugbank.targets (POST -d q=DB02527,DB14236,DB05382,DB01061,DB06174,DB01050,DB00605,DB01147,DB01416,DB06777,DB01378,DB01645,DB00137,DB14104,DB01067,DB00624,DB06211,DB00974,DB11457,DB14177,DB14522,DB00578,DB01160,DB00601,DB06709,DB00321,DB03600,DB15394,DB00594,DB00390,DB01045,DB13747,DB00158,DB00115,DB00927,DB00945,DB00783,DB15589,DB00893,DB09295,DB13867,DB00828,DB01708,DB00736,DB00456,DB00148,DB03128,DB09145,DB02362,DB00373,DB11672,DB00035,DB01592,DB00411,DB01110,DB00419,DB09140,DB08818,DB09419,DB01223,DB01409,DB00297,DB04173,DB01413,DB00146,DB11588,DB01032,DB00951,DB12598,DB06742,DB01112,DB00728,DB01004,DB14025,DB00355,DB01064,DB06149,DB01394,DB01603,DB12789,DB00201,DB02315,DB09157,DB00135,DB00999,DB05381,DB14135,DB00929,DB01165,DB00330,DB09344,DB01109,DB09110,DB05804,DB00563,DB00862,DB00367,DB14007,DB00435,DB00196,DB00080,DB01001,DB00003,DB00479,DB00859,DB01016,DB01174,DB01331,DB11576,DB00864,DB00531,DB0

API 17.1 EBIgene2phenotype API: 450 hits
API 16.1: https://biothings.ncats.io/semmedphenotype/query?fields=manifestation_of (POST -d q=C0006267,C3714497,C0023892,C0021051,C2169795,C0034069,C1856646,C3714745,C0159069,C0034917,C3806482,C0008312,C0542213,C0004096,C0441748,C4048270,C0034888,C4020899,C0086438,C0456984,C0021364,C0034072&scopes=umls)
API 16.1 SEMMED Phenotype API: 0 hits
API 16.2: https://biothings.ncats.io/semmedphenotype/query?fields=affected_by (POST -d q=C0006267,C3714497,C0023892,C0021051,C2169795,C0034069,C1856646,C3714745,C0159069,C0034917,C3806482,C0008312,C0542213,C0004096,C0441748,C4048270,C0034888,C4020899,C0086438,C0456984,C0021364,C0034072&scopes=umls)
API 16.2 SEMMED Phenotype API: 41 hits
API 16.3: https://biothings.ncats.io/semmedphenotype/query?fields=has_part (POST -d q=C0006267,C3714497,C0023892,C0021051,C2169795,C0034069,C1856646,C3714745,C0159069,C0034917,C3806482,C0008312,C0542213,C0004096,C0441748,C4048270,C0034888,C4020899,C0086438,C0456984,C0021364,C0

API 9.1 MGIgene2phenotype API: 457 hits


KeyboardInterrupt: 

## Top Genes -> Drugs 

In [103]:
# get gene inputs through hint module
gene_inputs = []
for gene in list(direct_gene_dict.keys()): 
    try: 
        gene_input = ht.query(gene)["Gene"][0]
        gene_inputs.append(gene_input)
    except: 
        print(gene + ' Failed')

print(gene_inputs)

[{'NCBIGene': '1080', 'name': 'CF transmembrane conductance regulator', 'SYMBOL': 'CFTR', 'UMLS': 'C1413365', 'HGNC': '1884', 'UNIPROTKB': 'P13569', 'ENSEMBL': 'ENSG00000001626', 'primary': {'identifier': 'NCBIGene', 'cls': 'Gene', 'value': '1080'}, 'display': 'NCBIGene(1080) ENSEMBL(ENSG00000001626) HGNC(1884) UMLS(C1413365) UNIPROTKB(P13569) SYMBOL(CFTR)', 'type': 'Gene'}, {'NCBIGene': '7040', 'name': 'transforming growth factor beta 1', 'SYMBOL': 'TGFB1', 'UMLS': 'C1366557', 'HGNC': '11766', 'UNIPROTKB': 'P01137', 'ENSEMBL': 'ENSG00000105329', 'primary': {'identifier': 'NCBIGene', 'cls': 'Gene', 'value': '7040'}, 'display': 'NCBIGene(7040) ENSEMBL(ENSG00000105329) HGNC(11766) UMLS(C1366557) UNIPROTKB(P01137) SYMBOL(TGFB1)', 'type': 'Gene'}, {'NCBIGene': '51164', 'name': 'dynactin subunit 4', 'SYMBOL': 'DCTN4', 'UMLS': 'C1423548', 'HGNC': '15518', 'UNIPROTKB': 'Q9UJW0', 'ENSEMBL': 'ENSG00000132912', 'primary': {'identifier': 'NCBIGene', 'cls': 'Gene', 'value': '51164'}, 'display': 'N

In [109]:
pd3 = Predict(
    input_objs=gene_inputs,
    intermediate_nodes= ['PhenotypicFeature'], 
    output_types =['ChemicalSubstance'], 
    config={"expand": True, "annotate":["drugPhase"]}
)
pd3.connect(verbose=True)


========== QUERY EXPANSION ==========

You have selected to expand your inputs.
No bioentity was found to be subclass of your inputs.

========== QUERY PARAMETER SUMMARY ==========

Your query have 20 input nodes, including CF transmembrane conductance regulator,transforming growth factor beta 1,dynactin subunit 4,homeostatic iron regulator,syntaxin 1A .... And BTE will find paths that connect your input nodes to your output types ['ChemicalSubstance']. Paths will contain 1 intermediate nodes.

Intermediate node #1 will have these type constraints: PhenotypicFeature

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Gene
Output Types: PhenotypicFeature
Predicates: None

BTE found 3 APIs based on SmartAPI Meta-KG.

API 1. BioLink API (20 API calls)
API 2. SEMMED Gene API (8 API calls)
API 3. EBIgene2phenotype API (1 API calls)


==== Step #2: Query path execution ====

NOTE: API requests are dispatched in parallel, so the list of APIs below is ordere

API 3.67 Automat CORD19 Scigraph API: 0 hits
API 8.69: https://automat.renci.org/pharos/gene/disease/NCBIGene:6337
API 8.69 Automat PHAROS API: 3 hits
API 5.67: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6340
API 5.67 Automat CORD19 Scibite API: 0 hits
API 5.68: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6337
API 5.68 Automat CORD19 Scibite API: 1 hits
API 3.68: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:22802
API 3.68 Automat CORD19 Scigraph API: 0 hits
API 3.69: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:6337
API 3.69 Automat CORD19 Scigraph API: 0 hits
API 5.69: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6804
API 5.69 Automat CORD19 Scibite API: 3 hits
API 9.65: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:22802/interactions?direct=True&rows=200&unselect_evidence=True
API 9.65 BioLink API: 0 hits
API 9.66: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGen

Task exception was never retrieved
future: <Task finished name='Task-32055' coro=<APIQueryDispatcher.asyncQuery() done, defined at /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py:383> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-105-c9b9b85e74b0>", line 7, in <module>
    pd3.connect(verbose=True)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/query/predict.py", line 159, in connect
    self.steps_results[i] = dp.syncQuery()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 411, in syncQuery
    return loop.run_until_complete(self.asyncQue

API 1.4 BioLink API: 84 hits
API 15.64: https://biothings.ncats.io/semmedgene/query?fields=related_to (POST -d q=C1366557,C1412179,C1420498,C1419868,C1384665,C1416825,C1420162,C1419869,C1337112,C1419871,C1413043,C1423548,C1413365,C1420259,C1413449,C1424562,C1412573,C1425842,C0919427,C1335836&scopes=umls)
API 15.64: https://biothings.ncats.io/semmedgene/query?fields=related_to (POST -d q=C1366557,C1412179,C1420498,C1419868,C1384665,C1416825,C1420162,C1419869,C1337112,C1419871,C1413043,C1423548,C1413365,C1420259,C1413449,C1424562,C1412573,C1425842,C0919427,C1335836&scopes=umls)
API 15.64: https://biothings.ncats.io/semmedgene/query?fields=positively_regulates (POST -d q=C1366557,C1412179,C1420498,C1419868,C1384665,C1416825,C1420162,C1419869,C1337112,C1419871,C1413043,C1423548,C1413365,C1420259,C1413449,C1424562,C1412573,C1425842,C0919427,C1335836&scopes=umls)
API 15.64: https://biothings.ncats.io/semmedgene/query?fields=prevents (POST -d q=C1366557,C1412179,C1420498,C1419868,C1384665,C14

API 5.76 Automat CORD19 Scibite API: 1 hits
API 8.77: https://automat.renci.org/pharos/gene/disease/NCBIGene:166785
API 8.77 Automat PHAROS API: 3 hits
API 5.77: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6609
API 5.77 Automat CORD19 Scibite API: 2 hits
API 5.78: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:166785
API 5.78 Automat CORD19 Scibite API: 0 hits
API 5.79: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:3952
API 5.79 Automat CORD19 Scibite API: 18 hits
API 8.78: https://automat.renci.org/pharos/gene/disease/NCBIGene:6609
API 8.78 Automat PHAROS API: 3 hits
API 8.79: https://automat.renci.org/pharos/gene/disease/NCBIGene:1836
API 8.79 Automat PHAROS API: 7 hits
API 9.75: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6609/interactions?direct=True&rows=200&unselect_evidence=True
API 9.75 BioLink API: 0 hits
API 1.10: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:4524/phenotypes?direct=True&rows

...

API 9.95 BioLink API: 26 hits
API 9.96: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6609/phenotypes?direct=True&rows=200&unselect_evidence=True
API 9.96: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3952/phenotypes?direct=True&rows=200&unselect_evidence=True
API 9.96 BioLink API: 25 hits
API 1.10: https://biothings.ncats.io/semmedphenotype/query?fields=negatively_regulates (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.10 SEMMED Phenotype API: 0 hits
API 9.97: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:197131/phenotypes?direct=True&rows=200&unselect_evidence=True
API 1.11: https://biothings.ncats.io/semmedphenotype/query?fields=physically_interacts_with (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.11 SEMMED Phenotype API: 0 hits
API 1.12: https://biothings.ncats.io/semmedphenotype/query?fields=part_of (PO

API 9.98 BioLink API: 71 hits
API 1.13 SEMMED Phenotype API: 1 hits
API 9.99: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1836/phenotypes?direct=True&rows=200&unselect_evidence=True
API 1.14 SEMMED Phenotype API: 243 hits
API 9.99 BioLink API: 149 hits
API 1.15: https://biothings.ncats.io/semmedphenotype/query?fields=positively_regulated_by (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.15 SEMMED Phenotype API: 0 hits
API 1.16: https://biothings.ncats.io/semmedphenotype/query?fields=positively_regulates (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.16 SEMMED Phenotype API: 0 hits
API 1.17: https://biothings.ncats.io/semmedphenotype/query?fields=produced_by (POST -d q=C0241772,C4020786,C1834055,C0917799,C0427064,C1837482,C4280712,C1873509,C0020490,C4025545,C0545053,C0020295,C4024613,C0575802,C0234182,C0234860,C0037932,C0020538,C0020305,C00855

API 9.100 BioLink API: 98 hits

BTE starts to perform id-to-object translation.

API 1.20: https://biothings.ncats.io/semmedphenotype/query?fields=produces (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.20 SEMMED Phenotype API: 0 hits
API 1.21: https://biothings.ncats.io/semmedphenotype/query?fields=related_to (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.21 SEMMED Phenotype API: 0 hits
API 1.22: https://biothings.ncats.io/semmedphenotype/query?fields=produced_by (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.22 SEMMED Phenotype API: 0 hits
API 1.23: https://biothings.ncats.io/semmedphenotype/query?fields=related_to (POST -d q=C0241772,C4020786,C1834055,C0917799,C0427064,C1837482,C4280712,C1873509,C0020490,C4025545,C0545053,C0020295,C4024613,C0575802,C0234182,C0234860,C0037932,C0020538,C0020305,C

API 1.25: https://biothings.ncats.io/semmedphenotype/query?fields=subclass_of (POST -d q=C0023530,C0011168,C0018824,C0029408,C0438237,C0339789,C0392188,C4020875,C0920163&scopes=umls)
API 1.25 SEMMED Phenotype API: 0 hits
API 1.26: https://biothings.ncats.io/semmedphenotype/query?fields=subclass_of (POST -d q=C0241772,C4020786,C1834055,C0917799,C0427064,C1837482,C4280712,C1873509,C0020490,C4025545,C0545053,C0020295,C4024613,C0575802,C0234182,C0234860,C0037932,C0020538,C0020305,C0085570,C0423250,C1842714,C0151563,C4023106,C0742906,C0399526,C1848701,C0004941,C1846345,C0747085,C1862496,C0151747,C1866210,C4023701,C0235475,C0424585,C4021768,C0086438,C2939175,C1855119,C4072858,C0206620,C0730362,C4022909,C0409348,C4280566,C0018418,C0008350,C0036857,C1866231,C4280644,C4022560,C0151686,C0149521,C0242350,C3714796,C0029456,C4025662,C3275754,C4073160,C0220982,C0042024,C0011813,C0011606,C4280612,C4023452,C4025675,C4280530,C4022738,C1263857,C0497327,C1290511,C3552463,C0034067,C0085680,C1856922,C00302

After id-to-object translation, BTE retrieved 775 unique output nodes.


Number of output edges sent to Drug Phase annotation is 1187. Number of output edges annotated with Drug Phase is 562

========== Final assembly of results ==========

In the #1 query, BTE found 781 unique nodes.
In the #2 query, BTE found 775 unique nodes.


In [111]:
df3 = pd3.display_table_view(extra_fields=["clinical_significance", "evidence_level"])
df3

,input_id,input_label,input_type,pred1,pred1_source,pred1_api,pred1_publications,node1_id,node1_label,node1_type,node1_degree,pred2,pred2_source,pred2_api,pred2_publications,output_id,output_label,output_type,output_degree
0,NCBIGene:3952,leptin,Gene,affects,SEMMED,SEMMED Gene API,PMID:16636130,UMLS:C0019699,UMLS:C0019699,PhenotypicFeature,1,affected_by,SEMMED,SEMMED Phenotype API,PMID:7683159,CHEBI:34795,ISOBUTYLMETHYLXANTHINE,ChemicalSubstance,1
1,NCBIGene:3952,leptin,Gene,affects,SEMMED,SEMMED Gene API,PMID:16636130,UMLS:C0019699,UMLS:C0019699,PhenotypicFeature,1,affected_by,SEMMED,SEMMED Phenotype API,"PMID:17487577,PMID:1939938,PMID:20574634,PMID:...",CHEBI:16236,ALCOHOL,ChemicalSubstance,8
2,NCBIGene:3952,leptin,Gene,affects,SEMMED,SEMMED Gene API,PMID:16636130,UMLS:C0019699,UMLS:C0019699,PhenotypicFeature,1,affected_by,SEMMED,SEMMED Phenotype API,PMID:8512745,MESH:D000975,Antioxidants,ChemicalSubstance,6
3,NCBIGene:3952,leptin,Gene,affects,SEMMED,SEMMED Gene API,PMID:16636130,UMLS:C0019699,UMLS:C0019699,PhenotypicFeature,1,affected_by,SEMMED,SEMMED Phenotype API,PMID:9689575,MESH:D000998,"Agents, Antiviral",ChemicalSubstance,1
4,NCBIGene:3952,leptin,Gene,affects,SEMMED,SEMMED Gene API,PMID:16636130,UMLS:C0019699,UMLS:C0019699,PhenotypicFeature,1,affected_by,SEMMED,SEMMED Phenotype API,"PMID:24568161,PMID:26195015",MESH:D007155,Biological Response Modifiers,ChemicalSubstance,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,NCBIGene:760,carbonic anhydrase 2,Gene,positively_regulated_by,SEMMED,SEMMED Gene API,"PMID:6138281,PMID:9051122",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,4,produces,SEMMED,SEMMED Phenotype API,PMID:2850425,CHEBI:18248,Iron,ChemicalSubstance,7
3758,NCBIGene:760,carbonic anhydrase 2,Gene,positively_regulated_by,SEMMED,SEMMED Gene API,"PMID:6138281,PMID:9051122",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,4,produces,SEMMED,SEMMED Phenotype API,"PMID:1911876,PMID:8216390",MESH:D019344,Lactate,ChemicalSubstance,2
3759,NCBIGene:760,carbonic anhydrase 2,Gene,positively_regulated_by,SEMMED,SEMMED Gene API,"PMID:6138281,PMID:9051122",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,4,subclass_of,SEMMED,SEMMED Phenotype API,"PMID:2116226,PMID:25532376,PMID:8546477",MESH:D004364,Pharmaceutical Preparations,ChemicalSubstance,16
3760,NCBIGene:760,carbonic anhydrase 2,Gene,positively_regulated_by,SEMMED,SEMMED Gene API,"PMID:6138281,PMID:9051122",UMLS:C0034917,UMLS:C0034917,PhenotypicFeature,4,subclass_of,SEMMED,SEMMED Phenotype API,"PMID:17932678,PMID:7385256",UMLS:C0220806,UMLS:C0220806,ChemicalSubstance,2
